In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "Electricity_311"
seq_length = 14
forecast_horizon = 7
model_class = "PINPOINT_2CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): Linear(in_features=64, out_features=7, bias=True)
  )
)

# Dataset

In [6]:
df = pd.read_csv(f"{working_directory}/data/Sensor_MT_311.csv", parse_dates=['Date'], index_col='Date')
df = df.iloc[:, 0]
df.index.freq = 'D'
entire_ts = df

train = entire_ts.loc[:pd.Timestamp("2014-01-01")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]

plot_name = "Consumption"
yaxis_name = "Daily mean (kW)"

train = train.append(validation)

print(train)
print(test)

Date
2011-01-01    125.260119
2011-01-02    150.060655
2011-01-03    141.936157
2011-01-04    147.133161
2011-01-05    148.731453
                 ...    
2014-02-20    114.129973
2014-02-21    116.113450
2014-02-22    120.734681
2014-02-23    116.681705
2014-02-24    112.345805
Freq: D, Name: MT_311, Length: 1151, dtype: float64
Date
2014-02-25    114.863651
2014-02-26    113.341554
2014-02-27    115.142264
2014-02-28    117.635547
2014-03-01    122.114112
                 ...    
2014-12-28    117.631336
2014-12-29    112.174166
2014-12-30    111.783967
2014-12-31    105.826616
2015-01-01     48.238691
Freq: D, Name: MT_311, Length: 311, dtype: float64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

expected_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [8]:
expected_output

Date
2014-02-25    116.585999
2014-02-26    115.911476
2014-02-27    116.587555
2014-02-28    117.372482
2014-03-01    117.978088
                 ...    
2014-12-28    117.307281
2014-12-29    116.424812
2014-12-30    117.386444
2014-12-31    115.458549
2015-01-01    113.785225
Freq: D, Length: 311, dtype: float64

In [9]:
print(experiment_name)
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

Electricity_311
MAE of model PINPOINT_2CONV, forecast horizon: 7: 5.76


## Encode the models

In [10]:
HE = Pyfhel()    
HE.contextGen(p=9615535171512, m=8192, intDigits=16, fracDigits=64) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.cpu().main)

# Encrypted processing

In [11]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)
    
    _min = HE.encodeFrac(scaler.min_[0])
    _scale = HE.encodeFrac(1.0 / scaler.scale_[0])
    
    encrypted_input = [list(map(lambda x: (x - _min)*_scale, encrypted_input[0]))]
    predict = decrypt_matrix(HE, encrypted_input)

    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [12]:
expected_output

Date
2014-02-25    116.585999
2014-02-26    115.911476
2014-02-27    116.587555
2014-02-28    117.372482
2014-03-01    117.978088
                 ...    
2014-12-28    117.307281
2014-12-29    116.424812
2014-12-30    117.386444
2014-12-31    115.458549
2015-01-01    113.785225
Freq: D, Length: 311, dtype: float64

In [13]:
decrypted_output

Date
2014-02-25    116.558833
2014-02-26    115.891106
2014-02-27    116.562804
2014-02-28    117.330512
2014-03-01    117.934110
                 ...    
2014-12-28    117.265692
2014-12-29    116.382962
2014-12-30    117.336423
2014-12-31    115.436340
2015-01-01    113.774654
Freq: D, Length: 311, dtype: float64

In [14]:
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

MAE of model PINPOINT_2CONV, forecast horizon: 7: 5.76
MAE of model PINPOINT_2CONV used on encrypted inputs, forecast horizon: 7: 5.75


Difference between expected and obtained on encrypted data:

In [15]:
print(expected_output - decrypted_output)

Date
2014-02-25    0.027165
2014-02-26    0.020370
2014-02-27    0.024751
2014-02-28    0.041970
2014-03-01    0.043978
                ...   
2014-12-28    0.041590
2014-12-29    0.041851
2014-12-30    0.050021
2014-12-31    0.022210
2015-01-01    0.010571
Freq: D, Length: 311, dtype: float64
